In [1]:
import re

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

# from itertools import pairwise

In [21]:
pd.__version__

'2.1.1'

In [2]:
import matplotlib.pyplot as plt

In [3]:
class Config:
    base_dir_path = f"data/"

    train_logs = f"{base_dir_path}train_logs.csv"
    train_scores = f"{base_dir_path}train_scores.csv"
    test_logs = f"{base_dir_path}test_logs.csv"

In [4]:
cnfg = Config()

In [5]:
train_logs_df = pd.read_csv(cnfg.train_logs)

In [6]:
train_logs_df.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [7]:
train_logs_df.id.nunique()

2471

In [8]:
train_logs_df = train_logs_df.sort_values(["id", "event_id"]).reset_index(drop=True)

In [9]:
# Function to construct essays copied from here: https://www.kaggle.com/code/kawaiicoderuwu/essay-contructor
def getEssays(df):
    textInputDf = df[["id", "activity", "cursor_position", "text_change"]]
    textInputDf = textInputDf[textInputDf.activity != "Nonproduction"]
    valCountsArr = textInputDf["id"].value_counts(sort=False).values
    lastIndex = 0
    essaySeries = pd.Series()
    for index, valCount in enumerate(valCountsArr):
        currTextInput = textInputDf[
            ["activity", "cursor_position", "text_change"]
        ].iloc[lastIndex : lastIndex + valCount]
        lastIndex += valCount
        essayText = ""
        for Input in currTextInput.values:
            if Input[0] == "Replace":
                replaceTxt = Input[2].split(" => ")
                essayText = (
                    essayText[: Input[1] - len(replaceTxt[1])]
                    + replaceTxt[1]
                    + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]) :]
                )
                continue
            if Input[0] == "Paste":
                essayText = (
                    essayText[: Input[1] - len(Input[2])]
                    + Input[2]
                    + essayText[Input[1] - len(Input[2]) :]
                )
                continue
            if Input[0] == "Remove/Cut":
                essayText = (
                    essayText[: Input[1]] + essayText[Input[1] + len(Input[2]) :]
                )
                continue
            if "M" in Input[0]:
                croppedTxt = Input[0][10:]
                splitTxt = croppedTxt.split(" To ")
                valueArr = [item.split(", ") for item in splitTxt]
                moveData = (
                    int(valueArr[0][0][1:]),
                    int(valueArr[0][1][:-1]),
                    int(valueArr[1][0][1:]),
                    int(valueArr[1][1][:-1]),
                )
                if moveData[0] != moveData[2]:
                    if moveData[0] < moveData[2]:
                        essayText = (
                            essayText[: moveData[0]]
                            + essayText[moveData[1] : moveData[3]]
                            + essayText[moveData[0] : moveData[1]]
                            + essayText[moveData[3] :]
                        )
                    else:
                        essayText = (
                            essayText[: moveData[2]]
                            + essayText[moveData[0] : moveData[1]]
                            + essayText[moveData[2] : moveData[0]]
                            + essayText[moveData[1] :]
                        )
                continue
            essayText = (
                essayText[: Input[1] - len(Input[2])]
                + Input[2]
                + essayText[Input[1] - len(Input[2]) :]
            )
        essaySeries[index] = essayText
    essaySeries.index = textInputDf["id"].unique()

    essay_df = (
        pd.DataFrame(essaySeries, columns=["essay"])
        .reset_index()
        .rename(columns={"index": "id"})
    )
    return essay_df

In [10]:
train_essays = getEssays(train_logs_df)

In [11]:
# train_essays = pd.DataFrame(train_essays,columns=['essay']).reset_index().rename(columns = {'index': 'id'})

In [12]:
train_essays

,id,essay
0,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...
1,0022f953,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq..."
2,0042269b,qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
3,0059420b,qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
4,0075873a,"qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq..."
...,...,...
2466,ffb8c745,"qq qqqqq'q qqqqqqq, qqq'q qqqqq q qqqq qq..."
2467,ffbef7e5,"qqqq qqqqqq qqqqq qq qqqqq qqqqq, qq qq q qqqq..."
2468,ffccd6fd,qqqqqq qqqq q qqqqqqq qqqqqqqqq qq qqqqqq qqqq...
2469,ffec5b38,"qqqqqqqqqq qqqqqqq, qqqqqq qqqq qqqqq qqqq qqq..."


In [13]:
def split_essays_into_sentences(df):
    essay_df = df
    essay_df["sent"] = essay_df["essay"].apply(lambda x: re.split("\\.|\\?|\\!", x))
    essay_df = essay_df.explode("sent")
    essay_df["sent"] = essay_df["sent"].apply(lambda x: x.replace("\n", "").strip())
    essay_df["sent_len"] = essay_df["sent"].apply(lambda x: len(x))
    essay_df = essay_df[essay_df.sent_len != 0].reset_index(drop=True)
    return essay_df

In [27]:
def split_essays_into_paragraphs(df):
    essay_df = df
    essay_df["paragraph"] = essay_df["essay"].apply(lambda x: x.split("\n"))
    essay_df = essay_df.explode("paragraph")
    essay_df["paragraph_len"] = essay_df["paragraph"].apply(lambda x: len(x))
    essay_df = essay_df[essay_df.paragraph_len != 0].reset_index(drop=True)
    return essay_df

In [14]:
sent_df = split_essays_into_sentences(train_essays)

In [15]:
sent_df.head()

,id,essay,sent,sent_len
0,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,qqqqqqqqq qq qqqqq qq qqqq qqqq,31
1,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,qqqqqq qqq qqqq qqqqqq qq qq qqqqq qq qqqq qqq...,107
2,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,qqqqqq qqq qqqqq qqq qqqqqqqqqqq qq qqq qqqqqq...,123
3,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,qqqqqqqq qq qqqqqqqqqq qqqq qqqq qqqqqqqqq qqq...,119
4,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...,qq qq qqqq qqqq qqq qqqqqqqqq qqq qqqqqqq qq q...,129


In [24]:
def compute_sentence_aggregations(df):
    sent_agg_df = (
        df[["id", "sent_len"]]
        .groupby(["id"])
        .agg(
            [
                "count",
                "mean",
                "std",
                "min",
                "max",
                "first",
                "last",
                "sem",
                q1,
                "median",
                q3,
                "skew",
                # pd.DataFrame.kurt,
            ]
        )
    )
    sent_agg_df.columns = ["_".join(x) for x in sent_agg_df.columns]
    sent_agg_df["id"] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    return sent_agg_df

In [30]:
def compute_paragraph_aggregations(df):
    paragraph_agg_df = (
        df[["id", "paragraph_len"]]
        .groupby(["id"])
        .agg(
            [
                "count",
                "mean",
                "std",
                "min",
                "max",
                "first",
                "last",
                "sem",
                q1,
                "median",
                q3,
                "skew",
                # pd.DataFrame.kurt,
            ]
        )
    )
    paragraph_agg_df.columns = ["_".join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df["id"] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    return paragraph_agg_df

In [31]:
train_sent_agg_df = compute_sentence_aggregations(sent_df)

In [32]:
train_sent_agg_df

,sent_len_count,sent_len_mean,sent_len_std,sent_len_min,sent_len_max,sent_len_first,sent_len_last,sent_len_sem,sent_len_q1,sent_len_median,sent_len_q3,sent_len_skew,id
0,14,106.142857,41.128050,31,196,31,89,10.991934,75.50,119.5,126.00,0.131863,001519c8
1,15,107.666667,64.713287,19,226,19,143,16.708899,56.50,92.0,149.50,0.508048,0022f953
2,19,133.842105,33.480115,73,189,139,161,7.680865,108.00,139.0,161.00,-0.220844,0042269b
3,13,86.846154,33.195999,39,144,99,80,9.206914,62.00,80.0,99.00,0.348766,0059420b
4,16,86.812500,44.094170,22,182,75,22,11.023543,60.00,74.0,106.25,1.031203,0075873a
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,13,121.076923,40.376275,55,180,79,109,11.198364,84.00,132.0,147.00,-0.000963,ffb8c745
2467,29,78.310345,40.481127,20,175,143,52,7.517157,52.00,67.0,105.00,0.860269,ffbef7e5
2468,4,277.000000,77.395090,200,359,223,359,38.697545,217.25,274.5,334.25,0.079121,ffccd6fd
2469,27,92.592593,33.747090,36,176,79,94,6.494631,63.00,98.0,105.50,0.552573,ffec5b38


In [33]:
train_paragraph_df = split_essays_into_paragraphs(train_essays)
train_paragraph_agg_df = compute_paragraph_aggregations(train_paragraph_df)

In [34]:
train_paragraph_agg_df

,paragraph_len_count,paragraph_len_mean,paragraph_len_std,paragraph_len_min,paragraph_len_max,paragraph_len_first,paragraph_len_last,paragraph_len_sem,paragraph_len_q1,paragraph_len_median,paragraph_len_q3,paragraph_len_skew,id
0,3,508.000000,134.208793,390,654,390,480,77.485483,435.00,480.0,567.00,0.897971,001519c8
1,6,278.166667,98.554384,176,462,240,284,40.234659,228.75,261.0,283.50,1.548347,0022f953
2,6,429.500000,101.087586,296,568,491,296,41.268834,356.75,444.5,483.50,-0.103767,0042269b
3,3,384.000000,56.471232,347,449,347,356,32.603681,351.50,356.0,402.50,1.682689,0059420b
4,5,283.400000,232.336609,23,627,351,23,103.904090,124.00,292.0,351.00,0.636813,0075873a
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,4,407.750000,91.790976,301,514,372,301,45.895488,354.25,408.0,461.50,-0.012990,ffb8c745
2467,6,387.166667,178.941797,144,648,144,228,73.052683,274.00,424.0,450.25,-0.011589,ffbef7e5
2468,3,918.333333,939.787387,327,2002,426,2002,542.586501,376.50,426.0,1214.00,1.710452,ffccd6fd
2469,5,509.600000,122.681702,380,672,672,380,54.864925,394.00,540.0,562.00,0.165516,ffec5b38


In [35]:
train_paragraph_agg_df.columns[train_paragraph_agg_df.isnull().any()].tolist()

['paragraph_len_std', 'paragraph_len_sem', 'paragraph_len_skew']

In [36]:
train_paragraph_agg_df[train_paragraph_agg_df.isna().any(axis=1)]

,paragraph_len_count,paragraph_len_mean,paragraph_len_std,paragraph_len_min,paragraph_len_max,paragraph_len_first,paragraph_len_last,paragraph_len_sem,paragraph_len_q1,paragraph_len_median,paragraph_len_q3,paragraph_len_skew,id
45,2,571.5,348.603643,325,818,325,818,246.5,448.25,571.5,694.75,NaN,044c5c54
55,2,1270.5,1214.102343,412,2129,412,2129,858.5,841.25,1270.5,1699.75,NaN,053be4e5
59,1,2290.0,NaN,2290,2290,2290,2290,NaN,2290.00,2290.0,2290.00,NaN,05f425a4
80,1,2426.0,NaN,2426,2426,2426,2426,NaN,2426.00,2426.0,2426.00,NaN,08288a1f
84,1,396.0,NaN,396,396,396,396,NaN,396.00,396.0,396.00,NaN,08e3c7ae
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331,1,1126.0,NaN,1126,1126,1126,1126,NaN,1126.00,1126.0,1126.00,NaN,f26e1dd2
2339,1,1160.0,NaN,1160,1160,1160,1160,NaN,1160.00,1160.0,1160.00,NaN,f30d37f9
2391,2,797.0,1125.713996,1,1593,1593,1,796.0,399.00,797.0,1195.00,NaN,f7551231
2417,1,1206.0,NaN,1206,1206,1206,1206,NaN,1206.00,1206.0,1206.00,NaN,faa72937
